# Using EcoFOCIpy to process raw field data

## NW1901 (NW Explorer)

## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

In [45]:
import yaml
import glob
import pandas as pd

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

In [46]:
sample_data_dir = '/Users/bell/ecoraid/2019/CTDcasts/nw1901/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [47]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
nutdatafile = sample_data_dir+'working/DiscreteNutrients/NW1901 Nutrient Data.txt' #<- point to cruise and process all files within
cruise_name = 'nw1901' #no hyphens
cruise_meta_file = sample_data_dir+'logs/NW1901.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpNutsWOCE.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/nw1901/rawconverted/ctd013.btl
Processing /

## Load csv Nutrient File

In [48]:
nut_data = pd.read_csv(nutdatafile,delimiter='\t')
nut_data

,Cast,Niskin,PO4 (uM),PO4 Flag,Sil (uM),Sil Flag,NO3 (uM),NO3 Flag,NO2 (uM),NO2 Flag,NH4 (uM),NH4 Flag
0,1,2,0.700,2,1.9,2,0.0,2,0.01,2,0.49,2
1,1,3,0.758,2,2.2,2,0.1,2,0.02,2,1.19,2
2,2,10,0.638,2,3.9,2,0.0,2,0.01,2,0.37,2
3,2,2,0.582,2,5.7,2,0.1,2,0.03,2,1.30,2
4,2,1,1.059,2,12.5,2,0.5,2,0.11,2,5.35,2
...,...,...,...,...,...,...,...,...,...,...,...,...
142,46,3,0.801,2,10.8,2,0.0,2,0.02,2,0.37,2
143,46,2,0.807,2,10.0,2,0.0,2,0.02,2,0.23,2
144,46,1,0.858,2,10.5,2,0.1,2,0.08,2,0.60,2
145,47,1,0.809,2,11.7,2,0.0,2,0.07,2,0.44,2


## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [49]:
keep_param = ['bottle','prdm']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(nut_data[nut_data['Cast']==matchcast],cdata.reset_index()[keep_param],right_on='bottle',left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
    except:
        continue

## Add Deployment meta information

In [50]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [51]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)

## Add institutional meta-information


In [52]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [53]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'Sil (uM)':'SI',
                            'PO4 (uM)':'PO4',
                            'NO2 (uM)':'NO2', 
                            'NO3 (uM)':'NO3',
                            'NH4 (uM)':'NH4',
                            'Sil Flag':'SI_WOCE_FLAG',
                            'PO4 Flag':'PO4_WOCE_FLAG',
                            'NO2 Flag':'NO2_WOCE_FLAG', 
                            'NO3 Flag':'NO3_WOCE_FLAG',
                            'NH4 Flag':'NH4_WOCE_FLAG',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history